In [ ]:
import xarray as xr
import fsspec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask.distributed import Client, progress
from pathlib import Path 

In [ ]:
client = Client()
client

In [ ]:
url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/precip.zarr'

In [ ]:
%%time
ds = xr.open_zarr(fsspec.get_mapper(url, anon=True), consolidated=True)

In [ ]:
ds

In [ ]:
ds.RAINRATE


In [ ]:
ds.RAINRATE.esri_pe_string

In [ ]:
## input the coordinates {y1:x1, y2:x2} or {lat1:long1, lat2:long2} format
Coordinates = {-1131207.186: 166599.7097} # Hobby Airport
##Coordinates = {-1115315.2193: 81203.1552,-1115178.9781: 73202.0731,-1114745.2073: 85011.1184,-1114823.3322: 77198.5283,-1110537.0666: 81155.3195,-1110497.0396: 85059.5592,-1110576.9827: 77065.1435,-1110111.0867: 69474.9337,-1110093.8945: 92771.3761,-1110030.4448: 88959.1182,-1110080.6105: 73341.9814,-1106436.4457: 77025.7806,-1106289.4588: 81205.7142,-1106220.0463: 65409.2229,-1105896.1206: 88356.2480,-1105632.8797: 92906.1487,-1105722.5089: 84730.7200,-1105731.1558: 72930.9845,-1102110.4083: 84971.5581,-1101497.2647: 92487.2548,-1100795.8328: 88764.9134}

In [ ]:
t=0
header = ''
for k, v in Coordinates.items():
    cord = str(k) + ',' + str(v)
    if (t==0):
        a = ds.RAINRATE.sel(y = k, x= v, method="nearest" )
        df = a.to_pandas()
        df =df.to_frame()
        df.rename(columns={0: cord}, inplace=True)
        
    else:
        a1 = ds.RAINRATE.sel(y = k, x= v, method="nearest" )
        df1 = a1.to_pandas()
        df1 =df1.to_frame()
        df1.rename(columns={0: cord}, inplace=True)
        
        df= pd.merge(df, df1, suffixes=(False, False), left_index=True, right_index=True )
    t+=1
    

In [ ]:
df

In [ ]:
filepath = Path('C:/Users/sbrady/Downloads/DickinsonBayou_NwmRetro2-1Zarr_Precip.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  